In [3]:
from pathlib import Path
import json
import os

from tinydb import TinyDB

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


In [7]:

    
class DocumentDB(object):
    def __init__(self, db_path):
        self._db_path = Path(db_path)
        self._db = None
        self._load_db()
        
    people_json = kv_data_dir.joinpath('people.json')
    visited_json = kv_data_dir.joinpath('visited.json')
    sites_json = kv_data_dir.joinpath('sites.json')
    measurements_json = kv_data_dir.joinpath('measurements.json')
    
    # load the individual json files into dict obj
    
    with open(sites_json) as f:
        _site_lookup = json.load(f)

    with open(people_json) as f:
        _person_lookup = json.load(f)
    
    
    with open(visited_json) as f:
        _visit_lookup = json.load(f)
    
    
    with open(measurements_json) as f:
        _measurements_lookup = json.load(f)
            
    def _get_site(self, site_id):
        return self._site_lookup[str(site_id)]
    
    def _get_measurements(self, person_id):
        measurements = []
        for values in self._measurements_lookup.values():
            if str(values['person_id']) == str(person_id):
                measurements.extend([values])
        return measurements
    
    def _get_visit(self, visit_id):
        visit = [v for k, v in self._visit_lookup.items() if v['visit_id']==visit_id][0]
        site_id = visit['site_id']
        site = self._get_site(site_id)
        visit['site'] = site
        return visit
    def _load_db(self):
        self._db = TinyDB(self._db_path)
        persons = self._person_lookup.items()
        for person_id, record in persons:
            # print('person record',record)
            measurements = self._get_measurements(person_id)
            visit_ids = set([measurement['visit_id'] for measurement in measurements])
            # print('unq vis ids',visit_ids)
            visits = []
            for visit_id in visit_ids:
                visit = self._get_visit(visit_id)
                # print('visit lookup',visit)
                visit['measurements'] = [measurement for measurement in measurements if visit_id == measurement['visit_id']]
                # print('after measuremnts',visit)
                visits.append(visit)
            record['visits'] = visits
            self._db.insert(record)

In [8]:
db_path = results_dir.joinpath('patient-info.json')
if db_path.exists():
    os.remove(db_path)

db = DocumentDB(db_path)